In [2]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import os
import cv2

#To run the prediction on the GPU, else comment out this line to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import scipy.misc
from csbdeep.utils.tf import limit_gpu_memory
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.path.append('../../Terminator/')
import imageio
from IPython.display import clear_output
import glob
from tifffile import imread
import tqdm
from csbdeep.utils import Path, normalize
from stardist import random_label_cmap, _draw_polygons,dist_to_coord
from stardist.models import StarDist2D
from TerminatorUtils.helpers import normalizeFloat, OtsuThreshold2D, save_8bit_tiff_imagej_compatible, save_16bit_tiff_imagej_compatible,Integer_to_border,SeedStarDistWatershed
from TerminatorUtils.helpers import Prob_to_Binary, zero_pad, SeedStarDistWatershedV2
from TerminatorUtils.helpers import save_tiff_imagej_compatible
from csbdeep.models import Config, CARE
np.random.seed(6)
lbl_cmap = random_label_cmap()

from skimage.morphology import remove_small_objects
from skimage.morphology import skeletonize
import png

import time

#from TerminatorUtils.helpers import limit_gpu_memory_actualised
#limit_gpu_memory_actualised(0.25)


Using TensorFlow backend.


**ROUND 1 + Model Import**

In [3]:
cutdir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated'        #where you need to cut
basedir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/Cut_Near_Midline/'   #where cur files are put
basedir_StardistWater_FijiMask = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/StarWat_FijiMask/'
basedir_StardistWater_FijiMask_RTGPipeline = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/StarWat_FijiMask_RTGPipeline/'

Model_Dir = '/home/sancere/NextonDisk_1/Lucas_Model_to_use/Segmentation/'  

In [4]:
Path(basedir).mkdir(exist_ok = True)
Raw_path = os.path.join(cutdir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'YX' # (0,1)
filesRaw = glob.glob(Raw_path)
filesRaw.sort


for fname in filesRaw:
   
    BigImage = imread(fname)
    SmallImage1 = BigImage[0:1816,0:BigImage.shape[1]]
    SmallImage2 = BigImage[1376:BigImage.shape[0],0:BigImage.shape[1]]
    
    HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
    HomeMadeMask1 = HomeMadeMask[0:1816,0:BigImage.shape[1]]
    HomeMadeMask2= HomeMadeMask[1376:BigImage.shape[0],0:BigImage.shape[1]]
        
    
    
    save_tiff_imagej_compatible((basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '.tif') , SmallImage1 , axes)
    imageio.imwrite(basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask1)
    save_tiff_imagej_compatible((basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImage2, axes)
    imageio.imwrite(basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask2)

 

In [5]:
StardistModelName = 'DrosophilaSegmentationSmartSeedsMatureMacroMiniElongatedLargePatch'

modelstar = StarDist2D(config = None, name = StardistModelName, basedir = Model_Dir)






Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.


In [6]:
Path(basedir_StardistWater_FijiMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

In [7]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)            
                     
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = SeedStarDistWatershedV2(prob,segmentationImage,modelstar.config.grid) 
            
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
        
            #Generation of masks                             
            HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            

            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
            StardistWater_FijiMask = np.logical_and(HomeMadeMask, Binary_Watershed[:originalX, :originalY])


            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = StardistWater_FijiMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_FijiMask = z 

            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_FijiMask, axes)


Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe2_mutant_s3_s4_s5_s6_OK_mov_0127.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe2_mutant_s3_s4_s5_s6_OK_mov_0054.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s4_s5_s6_OK_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe2_mutant_s3_s4_s5_s6_OK_mov_0010.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s3_s

In [8]:

import png

Path(basedir_StardistWater_FijiMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_FijiMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)
   
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_FijiMask_RTGPipeline + name.replace("StarWat_FijiMask_",'seg_') + '.png') 


**ROUND 2**

In [9]:
cutdir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated'
basedir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/Cut_Near_Midline/'
basedir_StardistWater_FijiMask = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/StarWat_FijiMask/'
basedir_StardistWater_FijiMask_RTGPipeline = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/StarWat_FijiMask_RTGPipeline/'

Model_Dir = '/home/sancere/NextonDisk_1/Lucas_Model_to_use/Segmentation/'  

In [10]:
Path(basedir).mkdir(exist_ok = True)
Raw_path = os.path.join(cutdir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort


for fname in filesRaw:
   
    BigImage = imread(fname)
    SmallImage1 = BigImage[0:1485,0:BigImage.shape[1]]
    SmallImage2 = BigImage[1193:BigImage.shape[0],0:BigImage.shape[1]]
    
    HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
    HomeMadeMask1 = HomeMadeMask[0:1485,0:BigImage.shape[1]]
    HomeMadeMask2= HomeMadeMask[1193:BigImage.shape[0],0:BigImage.shape[1]]
        
    
    
    save_tiff_imagej_compatible((basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '.tif') , SmallImage1 , axes)
    imageio.imwrite(basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask1)
    save_tiff_imagej_compatible((basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImage2, axes)
    imageio.imwrite(basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask2)

 

In [11]:
Path(basedir_StardistWater_FijiMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

In [12]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)            
                     
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = SeedStarDistWatershedV2(prob,segmentationImage,modelstar.config.grid) 
            
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
        
            #Generation of masks                             
            HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            

            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
            StardistWater_FijiMask = np.logical_and(HomeMadeMask, Binary_Watershed[:originalX, :originalY])


            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = StardistWater_FijiMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_FijiMask = z 

            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_FijiMask, axes)


Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe2_mutant_s5_s6_s7_s8_mov_0157.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe2_mutant_s5_s6_s7_s8_mov_0035.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe2_mutant_s5_s6_s7_s8_mov_0053.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe2_mutant_s5_s6_s7_s8_mov/Rotate

In [13]:

import png

Path(basedir_StardistWater_FijiMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_FijiMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)

   
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_FijiMask_RTGPipeline + name.replace("StarWat_FijiMask_",'seg_') + '.png') 


**ROUND 3**

In [24]:
cutdir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated'
basedir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/Cut_Near_Midline/'
basedir_StardistWater_FijiMask = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/StarWat_FijiMask/'
basedir_StardistWater_FijiMask_RTGPipeline = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/StarWat_FijiMask_RTGPipeline/'

Model_Dir = '/home/sancere/NextonDisk_1/Lucas_Model_to_use/Segmentation/'  

In [25]:
Path(basedir).mkdir(exist_ok = True)
Raw_path = os.path.join(cutdir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort


for fname in filesRaw:
   
    BigImage = imread(fname)
    SmallImage1 = BigImage[0:1593,0:BigImage.shape[1]]
    SmallImage2 = BigImage[1278:BigImage.shape[0],0:BigImage.shape[1]]
    
    HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
    HomeMadeMask1 = HomeMadeMask[0:1593,0:BigImage.shape[1]]
    HomeMadeMask2= HomeMadeMask[1278:BigImage.shape[0],0:BigImage.shape[1]]
        
    
    
    save_tiff_imagej_compatible((basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '.tif') , SmallImage1 , axes)
    imageio.imwrite(basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask1)
    save_tiff_imagej_compatible((basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImage2, axes)
    imageio.imwrite(basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask2)

 

In [26]:
Path(basedir_StardistWater_FijiMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

In [27]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)            
                     
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = SeedStarDistWatershedV2(prob,segmentationImage,modelstar.config.grid) 
            
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
        
            #Generation of masks                             
            HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            

            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
            StardistWater_FijiMask = np.logical_and(HomeMadeMask, Binary_Watershed[:originalX, :originalY])


            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = StardistWater_FijiMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_FijiMask = z 

            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_FijiMask, axes)


Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe3_s7_s8_s9_s10_mov_0159.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe3_s7_s8_s9_s10_mov_0105.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe3_s7_s8_s9_s10_mov_0020.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe3_s7_s8_s9_s10_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe3_s7_s8_

In [28]:

import png

Path(basedir_StardistWater_FijiMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_FijiMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)


   
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_FijiMask_RTGPipeline + name.replace("StarWat_FijiMask_",'seg_') + '.png') 


In [29]:
from csbdeep.utils import Path

TriggerName = '/home/sancere/NextonDisk_1/TimeTrigger/TT2'
Path(TriggerName).mkdir(exist_ok = True)

**ROUND 4**

In [19]:
cutdir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated'
basedir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/Cut_Near_Midline/'
basedir_StardistWater_FijiMask = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/StarWat_FijiMask/'
basedir_StardistWater_FijiMask_RTGPipeline = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/StarWat_FijiMask_RTGPipeline/'

Model_Dir = '/home/sancere/NextonDisk_1/Lucas_Model_to_use/Segmentation/'  

In [20]:
Path(basedir).mkdir(exist_ok = True)
Raw_path = os.path.join(cutdir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort


for fname in filesRaw:
   
    BigImage = imread(fname)
    SmallImage1 = BigImage[0:1727,0:BigImage.shape[1]]
    SmallImage2 = BigImage[1424:BigImage.shape[0],0:BigImage.shape[1]]
    
    HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
    HomeMadeMask1 = HomeMadeMask[0:1727,0:BigImage.shape[1]]
    HomeMadeMask2= HomeMadeMask[1424:BigImage.shape[0],0:BigImage.shape[1]]
        
    
    
    save_tiff_imagej_compatible((basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '.tif') , SmallImage1 , axes)
    imageio.imwrite(basedir + 'P1_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask1)
    save_tiff_imagej_compatible((basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0])+ '.tif' ) , SmallImage2, axes)
    imageio.imwrite(basedir + 'P2_' + os.path.basename(os.path.splitext(fname)[0]) + '_Mask.png', HomeMadeMask2)

 

In [21]:
Path(basedir_StardistWater_FijiMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

In [22]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)            
                     
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = SeedStarDistWatershedV2(prob,segmentationImage,modelstar.config.grid) 
            
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
        
            #Generation of masks                             
            HomeMadeMask = cv2.imread(fname.replace('.tif','_Mask.png'))
            HomeMadeMask = HomeMadeMask[:,:,0]
            

            #Create StardistWater_StardistMask and StardistWater_UnetMask with logical and operation            
            StardistWater_FijiMask = np.logical_and(HomeMadeMask, Binary_Watershed[:originalX, :originalY])


            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = StardistWater_FijiMask 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_FijiMask = z 

            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_FijiMask + 'StarWat_FijiMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_FijiMask, axes)


Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe4_mutant_s13_s14_s15_s16_mov_0031.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/Cut_Near_Midline/P2_Rotated_pupe4_mutant_s13_s14_s15_s16_mov_0093.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutant_s13_s14_s15_s16_mov/Rotated/Cut_Near_Midline/P1_Rotated_pupe4_mutant_s13_s14_s15_s16_mov_0023.tif
Processing file /run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/mf_di-pietro/Movies-afterCARE/RhoGEF4 mutant/Movies_To_Segment/pupe4_mutan

In [23]:

import png

Path(basedir_StardistWater_FijiMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_FijiMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)

    
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_FijiMask_RTGPipeline + name.replace("StarWat_FijiMask_",'seg_') + '.png') 
